# <img src="https://files.training.databricks.com/images/DeltaLake-logo.png" width=80px> Managed Delta Lake

Delta Lake&reg; managed and queried via the Databricks platform includes additional features and optimizations.

These include:

- **Optimize**

- **Data skipping**

- **Z-Order**

- **Caching**

<img src="https://www.evernote.com/l/AAGv1SuWeRNJM4TI4bIOyGNPm0CTHa17PLwB/image.png" width=900px>

### Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run ./Includes/Classroom-Setup

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Created user-specific database

Using the database vanitha_a_sivakumar_gmail_com_db .

All done!

In [0]:
%run ./Includes/Delta-Optimization-Setup

In [0]:
spark.sql("""
    DROP TABLE IF EXISTS iot_data
  """)
spark.sql("""
    CREATE TABLE iot_data
    USING DELTA
    LOCATION '{}/delta/iot-events/'
  """.format(userhome))

Out[6]: DataFrame[]

Set up relevant paths.

In [0]:
iotPath = userhome + "/delta/iot-events/"

## SMALL FILE PROBLEM

Historical and new data is often written in very small files and directories.

This data may be spread across a data center or even across the world (that is, not co-located).

The result is that a query on this data may be very slow due to
* network latency
* volume of file metatadata

The solution is to compact many small files into one larger file.
Delta Lake has a mechanism for compacting small files.

-sandbox
### OPTIMIZE
Delta Lake supports the `OPTIMIZE` operation, which performs file compaction.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> Small files are compacted together into new larger files up to 1GB.
Thus, at this point the number of files increases!

The 1GB size was determined by the Databricks optimization team as a trade-off between query speed and run-time performance when running Optimize.

`OPTIMIZE` is not run automatically because you must collect many small files first.

* Run `OPTIMIZE` more often if you want better end-user query performance
* Since `OPTIMIZE` is a time consuming step, run it less often if you want to optimize cost of compute hours
* To start with, run `OPTIMIZE` on a daily basis (preferably at night when spot prices are low), and determine the right frequency for your particular business case
* In the end, the frequency at which you run `OPTIMIZE` is a business decision

The easiest way to see what `OPTIMIZE` does is to perform a simple `count(*)` query before and after and compare the timing!

Take a look at the `iotPath + "/date=2018-06-01/" ` directory.

Notice, in particular files like `../delta/iot-events/date=2018-07-26/part-xxxx.snappy.parquet`. There are hundreds of small files!

In [0]:
display(dbutils.fs.ls(iotPath + "/date=2016-07-26")) #contains 200 small files

path,name,size
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events/date=2016-07-26/part-00000-3c708b26-7685-49a6-afca-91d977bc19ac.c000.snappy.parquet,part-00000-3c708b26-7685-49a6-afca-91d977bc19ac.c000.snappy.parquet,2607
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events/date=2016-07-26/part-00001-38eaa918-efc2-4902-b962-011658d96723.c000.snappy.parquet,part-00001-38eaa918-efc2-4902-b962-011658d96723.c000.snappy.parquet,2604
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events/date=2016-07-26/part-00002-5333894b-36ca-4bca-b476-7634d96421f2.c000.snappy.parquet,part-00002-5333894b-36ca-4bca-b476-7634d96421f2.c000.snappy.parquet,2559
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events/date=2016-07-26/part-00003-f0db46d2-25b3-4806-ab40-e9f173adfef4.c000.snappy.parquet,part-00003-f0db46d2-25b3-4806-ab40-e9f173adfef4.c000.snappy.parquet,2601
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events/date=2016-07-26/part-00004-3b312d87-f06d-4c2e-83dc-3e18f9f9bb74.c000.snappy.parquet,part-00004-3b312d87-f06d-4c2e-83dc-3e18f9f9bb74.c000.snappy.parquet,2536
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events/date=2016-07-26/part-00005-22fff01f-1f44-44de-9331-ace3067bdc37.c000.snappy.parquet,part-00005-22fff01f-1f44-44de-9331-ace3067bdc37.c000.snappy.parquet,2642
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events/date=2016-07-26/part-00006-9107b1c0-cb55-42ac-881d-2d0d2b095779.c000.snappy.parquet,part-00006-9107b1c0-cb55-42ac-881d-2d0d2b095779.c000.snappy.parquet,2715
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events/date=2016-07-26/part-00007-dd70171b-5358-4242-bb2d-1c27971fd0c9.c000.snappy.parquet,part-00007-dd70171b-5358-4242-bb2d-1c27971fd0c9.c000.snappy.parquet,2633
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events/date=2016-07-26/part-00008-57b5530f-d44d-4d67-9b17-876bf4dc63a3.c000.snappy.parquet,part-00008-57b5530f-d44d-4d67-9b17-876bf4dc63a3.c000.snappy.parquet,2631
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events/date=2016-07-26/part-00009-ab73092a-1b11-4e5e-a48b-f18c8c16e537.c000.snappy.parquet,part-00009-ab73092a-1b11-4e5e-a48b-f18c8c16e537.c000.snappy.parquet,2694


CAUTION: Run this query. Notice it is very slow, due to the number of small files.

In [0]:
%sql
SELECT * FROM iot_data where deviceId=92 -- more than a minute to run in databricks community edition

action,time,date,deviceId
Close,1469579759,2016-07-27,92
Open,1469582631,2016-07-27,92
Open,1469604839,2016-07-27,92
Open,1469598785,2016-07-27,92
Open,1469609185,2016-07-27,92
Open,1469620945,2016-07-27,92
Close,1469653368,2016-07-27,92
Open,1469604867,2016-07-27,92
Close,1469650770,2016-07-27,92
Close,1469603427,2016-07-27,92


-sandbox
### Data Skipping and ZORDER

Delta Lake uses two mechanisms to speed up queries.

<b>Data Skipping</b> is a performance optimization that aims at speeding up queries that contain filters (WHERE clauses).

For example, we have a data set that is partitioned by `date`.

A query using `WHERE date > 2016-07-26` would not access data that resides in partitions that correspond to dates prior to `2016-07-26`.

<b>ZOrdering</b> is a technique to colocate related information in the same set of files.

ZOrdering maps multidimensional data to one dimension while preserving locality of the data points.

Given a column that you want to perform ZORDER on, say `OrderColumn`, Delta
* takes existing parquet files within a partition
* maps the rows within the parquet files according to `OrderColumn` using the algorithm described <a href="https://en.wikipedia.org/wiki/Z-order_curve" target="_blank">here</a>
* (in the case of only one column, the mapping above becomes a linear sort)
* rewrites the sorted data into new parquet files

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> You cannot use the partition column also as a ZORDER column.

#### ZORDER Technical Overview

A brief example of how this algorithm works (refer to [this blog](https://databricks.com/blog/2018/07/31/processing-petabytes-of-data-in-seconds-with-databricks-delta.html) for more details):

![](https://files.training.databricks.com/images/adbcore/zorder.png)

Legend:
- Gray dot = data point e.g., chessboard square coordinates
- Gray box = data file; in this example, we aim for files of 4 points each
- Yellow box = data file that’s read for the given query
- Green dot = data point that passes the query’s filter and answers the query
- Red dot = data point that’s read, but doesn’t satisfy the filter; “false positive”

-sandbox


#### ZORDER example
In the image below, table `Students` has 4 columns:
* `gender` with 2 distinct values
* `Pass-Fail` with 2 distinct values
* `Class` with 4 distinct values
* `Student` with many distinct values

Suppose you wish to perform the following query:

```SELECT Name FROM Students WHERE gender = 'M' AND Pass_Fail = 'P' AND Class = 'Junior'```

```ORDER BY Gender, Pass_Fail```

The most effective way of performing that search is to order the data starting with the largest set, which is `Gender` in this case.

If you're searching for `gender = 'M'`, then you don't even have to look at students with `gender = 'F'`.

Note that this technique only works if all `gender = 'M'` values are co-located.


<div><img src="https://files.training.databricks.com/images/eLearning/Delta/zorder.png" style="height: 300px"/></div><br/>

#### ZORDER usage

With Delta Lake the notation is:

> `OPTIMIZE Students`<br>
`ZORDER BY Gender, Pass_Fail`

This will ensure all the data backing `Gender = 'M' ` is colocated, then data associated with `Pass_Fail = 'P' ` is colocated.

See References below for more details on the algorithms behind ZORDER.

Using ZORDER, you can order by multiple columns as a comma separated list; however, the effectiveness of locality drops.

In streaming, where incoming events are inherently ordered (more or less) by event time, use `ZORDER` to sort by a different column, say 'userID'.

In [0]:
%sql
OPTIMIZE iot_data
ZORDER by (deviceId) --sorts the data and write a new parquet file combining smaller files

path,metrics
null,"List(3, 600, List(64649, 260297, 182389.0, 3, 547169), List(1368, 2975, 2303.0, 600, 1381862), 3, List(minCubeSize(107374182400), List(0, 0), List(600, 1381862), 0, List(600, 1381862), 3, null), 1, 600, 0, false)"


In [0]:
%sql
SELECT * FROM iot_data WHERE deviceId=92 --command took less than 3 secs now

action,time,date,deviceId
Close,1469579759,2016-07-27,92
Open,1469582631,2016-07-27,92
Open,1469604839,2016-07-27,92
Open,1469598785,2016-07-27,92
Open,1469609185,2016-07-27,92
Open,1469620945,2016-07-27,92
Close,1469653368,2016-07-27,92
Open,1469604867,2016-07-27,92
Close,1469650770,2016-07-27,92
Close,1469603427,2016-07-27,92


-sandbox
## VACUUM

To save on storage costs you should occasionally clean up invalid files using the `VACUUM` command.

Invalid files are small files compacted into a larger file with the `OPTIMIZE` command.

The  syntax of the `VACUUM` command is
>`VACUUM name-of-table RETAIN number-of HOURS;`

The `number-of` parameter is the <b>retention interval</b>, specified in hours.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> Databricks does not recommend you set a retention interval shorter than seven days because old snapshots and uncommitted files can still be in use by concurrent readers or writers to the table.

The scenario here is:
0. User A starts a query off uncompacted files, then
0. User B invokes a `VACUUM` command, which deletes the uncompacted files
0. User A's query fails because the underlying files have disappeared

Invalid files can also result from updates/upserts/deletions.

More details are provided here: <a href="https://docs.databricks.com/delta/optimizations.html#garbage-collection" target="_blank"> Garbage Collection</a>.

In [0]:
len(dbutils.fs.ls(iotPath + "date=2016-07-26")) #201 files: 200(small files) + 1 new file created using zorder

Out[9]: 201

-sandbox
<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> In the example below we set off an immediate `VACUUM` operation with an override of the retention check so that all files are cleaned up immediately.

Do not do this in production!

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> If using Databricks Runtime 5.1, in order to use a retention time of 0 hours, the following flag must be set.

In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

In [0]:
%sql

VACUUM iot_data RETAIN 0 HOURS; --dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events

path
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/iot-events


-sandbox
<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Notice how the directory looks vastly cleaned up!

In [0]:
len(dbutils.fs.ls(iotPath + "date=2016-07-26")) # output 1
# all the small files (200) are removed

Out[11]: 1

## Summary
Delta Lake offers key features that allow for query optimization and garbage collection, resulting in improved performance.

## Additional Topics & Resources

* <a href="https://docs.databricks.com/delta/optimizations.html#" target="_blank">Optimizing Performance and Cost</a>
* <a href="http://parquet.apache.org/documentation/latest/" target="_blank">Parquet Metadata</a>
* <a href="https://en.wikipedia.org/wiki/Z-order_curve" target="_blank">Z-Order Curve</a>